## side project: replace collected altitude data with data from openelevation API
reason: the collected altitude data is not very accurate

to consider:
- dump data into json file once 1000 entries have been crawled, so it is reasonably fast
- 45 entries can be requested from open-elevation at once
- openelevation api is very slow, esp. in the evening

In [4]:
import json

def prepare_location_history(json_file): # taken from main
    '''takes google location history and formats it so its data points can be evaluated'''
    # open and read json file into info:
    with open(json_file) as f:
        info = f.read()
    info = json.loads(info)
    info = info["locations"] 
    info = info[0::10] # NEEDS TO BE REMOVED ONCE COMPLETED. ONLY FOR QUICKNESS WHILE WORKING ON IT
    return info

In [5]:
info = prepare_location_history("standortverlauf_m_iran.json")

with open("altitudes.json", mode='w', encoding='utf-8') as f: # initialize empty list in json file
    json.dump([], f)

## find extreme altitudes

In [9]:
from geopy.geocoders import Nominatim
import requests

# lowest point that can potentially be reached (caspian depression, kazakhstan)
min_alt = -138

def find_extreme_altitudes(info):
    '''creates list with extreme altitude points from info'''
    newalts = []
    for entry in info:
        if "altitude" in entry:
            if entry["altitude"] <= min_alt:
                newalts.append(0)
            else:
                newalts.append(entry["altitude"])
        else:
            newalts.append(0)
    newalts.sort()
    return newalts[:10], newalts[len(newalts) - 10:] # returns one list with two lists in it - mins and maxs

minsmaxs = find_extreme_altitudes(info)
mins = minsmaxs[0]
maxs = minsmaxs[1]

In [10]:
def find_coordinates_by_altitude(altitude):
    '''finds all entries in dict that match a given altitude'''
    coordinates_that_match_altitude = [item for item in info if "altitude" in item if item["altitude"] == altitude]
    return coordinates_that_match_altitude

minpoint = find_coordinates_by_altitude(mins[0])[0]
maxpoint = find_coordinates_by_altitude(maxs[-1])[0]

In [11]:
def localize_dict_entry(entry):
    '''returns address of a dict entry in google maps location json file'''
    geolocator = Nominatim(user_agent="find_place")
    geocoordinates = entry["latitudeE7"] / 10**7, entry["longitudeE7"] / 10**7
    location = geolocator.reverse(geocoordinates)
    return location.address

lowest_point_address = localize_dict_entry(minpoint)
highest_point_address = localize_dict_entry(maxpoint)

In [12]:
def print_max_and_min(lowest_point_address, highest_point_address):
    print("The highest point volvo reached so far is %s at %s meters." %(highest_point_address, maxpoint["altitude"]))
    print("The lowest point volvo reached so far is %s at %s meters." %(lowest_point_address, minpoint["altitude"]))

print_max_and_min(lowest_point_address, highest_point_address)

The highest point volvo reached so far is Erzurum-Bingöl yolu, Kızılca, Aşağıçatköy, Çat, Erzurum, Doğu Anadolu Bölgesi, Türkiye at 2203 meters.
The lowest point volvo reached so far is گمرک مرزی آستارا, AH8, ویرمونی, بخش مرکزی, شهرستان آستارا, استان گیلان, ‏ایران‎ at -128 meters.


the lowest point is unrealistic; it's an outlier.

# CHECK ACCURACY OF ALTITUDES END EDIT THEM IN JSON FILE


In [13]:
def get_altitude_from_open_elevation(latitude, longitude):
    '''returns open elevation altitude as dict so it can be compared with google maps altitude
    after that, dict is ready for being tested against measured value'''
    a = requests.get("https://api.open-elevation.com/api/v1/lookup?locations={0},{1}".format(latitude / 10**7, longitude / 10**7))
    a = a.text
    a = json.loads(a)
    return a["results"][0]

#adjust functions so 40 points can be searched???
def get_40_altitudes_from_open_elevation(latitude, longitude):
    '''returns open elevation altitude as dict so it can be compared with google maps altitude
    after that, dict is ready for being tested against measured value'''
    a = requests.get("https://api.open-elevation.com/api/v1/lookup?locations={0},{1}".format(latitude / 10**7, longitude / 10**7))
    a = a.text
    a = json.loads(a)
    return a["results"][0]

In [14]:
a = get_altitude_from_open_elevation(maxpoint["latitudeE7"], maxpoint["longitudeE7"])
print(a)

{'latitude': 39.6656749, 'elevation': 2160, 'longitude': 41.0132522}


In [24]:
def is_true_value(point):
    '''finally: tests whether a google altitude is accurate (by 5 meters).'''
    a = get_altitude_from_open_elevation(point["latitudeE7"], point["longitudeE7"])
    
    #b = prepare_json_file_for_outlier_test(a) #dict is ready for being tested against measured value
    c = abs(point["altitude"] - a["elevation"])
    
    if c <= 5:
        return True
    else:
        print("Altitude was wrong by {} meters. Google maps data needs to be corrected.".format(c)) #goal: automatically correct data

In [22]:
bb = is_true_value(maxpoint)

Altitude was wrong by 43 meters. Google maps data needs to be corrected.


# REPLACE ALTITUDE VALUES IN JSON FILE (work in progress)

In [23]:
def change_altitude(maxpoint, b, info):
    for i, e in enumerate(info): # searches dict in list of dicts (by timestamp, because it is the unique value) and replaces altitude 
        if e["timestampMs"] == maxpoint["timestampMs"]:
            e["altitude"] = b["elevation"]
            print(i, e)
            info[i] = e
    
# put it in separate function so many values can be replaced before writing to file.
def write_altitude_changes_to_file(info):
    with open("testdata_map.json", mode='w', encoding='utf-8') as feedsjson:
        json.dump(info, feedsjson)